VACATIONPY

STARTER CODE TO IMPORT LIBRARIES AND LOAD THE WEATHER AND COORDINATES DATA

In [1]:
# CELL 1 OF HOMEWORK
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# CELL 2 OF HOMEWORK
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_hw.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,276.96,81,100,4.63,AR,1714977040
1,1,belyy yar,53.6039,91.3903,285.09,87,40,2.00,RU,1714977041
2,2,antsiranana,-12.2787,49.2917,302.57,54,20,6.17,MG,1714977041
3,3,aasiaat,68.7098,-52.8699,259.74,95,54,3.28,GL,1714977041
4,4,bubaque,11.2833,-15.8333,299.06,67,34,4.25,GW,1714977041
5,5,yellowknife,62.4560,-114.3525,279.49,79,20,5.66,CA,1714977041
6,6,port-aux-francais,-49.3500,70.2167,278.61,82,61,12.09,TF,1714977041
7,7,zhigalovo,54.8097,105.1578,298.63,24,83,5.71,RU,1714977042
8,8,klyuchi,52.2667,79.1667,284.19,33,6,6.05,RU,1714977042
9,9,praya,14.9215,-23.5087,296.45,60,20,4.12,CV,1714977042


STEP 1: CREATE A MAP THAT DISPLAYS A POINT FOR EVERY CITY IN THE CITY_DATA_DF DATAFRAME. THE SIZE OF THE POINT SHOULD BE THE HUMIDITY IN EACH CITY.

In [3]:
# CELL 3 OF HOMEWORK


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width =500,
    frame_height = 300,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    tiles="OSM"
)
# Display the map
map_plot_1

C:\Users\ramir\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

STEP 2: NARROW DOWN THE CITY_DATA_DF DATAFRAME TO FIND YOUR IDEAL WEATHER CONDITION

In [4]:
# CELL 4 OF HOMEWORK
# Narrow down cities that fit criteria and drop any results with null values
ideal=city_data_df[(city_data_df["Max Temp"]<300)&(city_data_df["Max Temp"]>295)&(city_data_df["Wind Speed"]<3)&(city_data_df["Cloudiness"]<10)]


# Drop any rows with null values
ideal.dropna()

# Display sample data
ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,palmas,-10.2128,-48.3603,299.08,73,0,2.57,BR,1714977046
151,151,chisec,15.8167,-90.2833,295.96,72,9,0.66,GT,1714977070
210,210,santa pola,38.1917,-0.5658,296.68,58,0,1.54,ES,1714977082
211,211,san pedro pochutla,15.7333,-96.4667,298.91,66,0,1.44,MX,1714977083
213,213,sinazongwe,-17.2614,27.4618,299.52,44,0,2.05,ZM,1714977083


In [5]:
len(ideal)

12

STEP 3: CREATE A NEW DATAFRAME CALLED HOTEL_DF

In [6]:
# CELL 5 OF HOMEWORK
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
28,palmas,BR,-10.2128,-48.3603,73,
151,chisec,GT,15.8167,-90.2833,72,
210,santa pola,ES,38.1917,-0.5658,58,
211,san pedro pochutla,MX,15.7333,-96.4667,66,
213,sinazongwe,ZM,-17.2614,27.4618,44,
240,baiti,SN,13.7450,-14.7482,25,
347,chiredzi,ZW,-21.0500,31.6667,39,
365,port saint john's,ZA,-31.6229,29.5448,36,
386,richards bay,ZA,-28.7830,32.0377,41,
483,san luis de la loma,MX,17.2706,-100.8939,78,


STEP 4: FOR EACH CITY, USE THE GEOAPIFY API TO FIND THE FIRST HOTEL LOCATED WITHIN 10,000 METRES OF YOUR COORDINATES

In [7]:
# CELL 6 OF HOMEWORK
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
           "apiKey":geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng=row["Lng"]
    lat=row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address=name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmas - nearest hotel: Hotel Triângulo Mineiro
chisec - nearest hotel: bombil pek
santa pola - nearest hotel: Quatre Llunes
san pedro pochutla - nearest hotel: Olimpia
sinazongwe - nearest hotel: Lake View Lodge
baiti - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel
port saint john's - nearest hotel: Outback Inn
richards bay - nearest hotel: SUN1 Richards Bay
san luis de la loma - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate


,City,Country,Lat,Lng,Humidity,Hotel Name
28,palmas,BR,-10.2128,-48.3603,73,Hotel Triângulo Mineiro
151,chisec,GT,15.8167,-90.2833,72,bombil pek
210,santa pola,ES,38.1917,-0.5658,58,Quatre Llunes
211,san pedro pochutla,MX,15.7333,-96.4667,66,Olimpia
213,sinazongwe,ZM,-17.2614,27.4618,44,Lake View Lodge
240,baiti,SN,13.7450,-14.7482,25,No hotel found
347,chiredzi,ZW,-21.0500,31.6667,39,Nesbitt Arms Hotel
365,port saint john's,ZA,-31.6229,29.5448,36,Outback Inn
386,richards bay,ZA,-28.7830,32.0377,41,SUN1 Richards Bay
483,san luis de la loma,MX,17.2706,-100.8939,78,No hotel found


STEP 5: ADD THE HOTEL NAME AND THE COUNTRY AS ADDITIONAL INFORMATION IN THE HOVER MESSAGE FOR EACH CITY IN THE MAP

In [16]:
#CELL 7 OF HOMEWORK
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width =500,
    frame_height = 300,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    tiles="OSM",
    hover_cols=["Country","Hotel Name"])
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)